# Введение в обработку естественного языка

В данном мини-проекте я попробую создать чат-бота с нуля.

Бот в телеграме: @Nlp_course_2022_test

In [16]:
import random
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import json

In [17]:
BOT_CONFIG = {
    'intents': {
        'hello': {
            'examples': ['Привет','Добрый день','Привет, бот', 'нихао'],
            'responses': ['Привет, человек','И вам здравствуйте', 'Доброго времени суток']
        },
        
        'bye': {
             'examples': ['Пока', 'До свидания', 'Досвидания', 'До скорой встречи'],
            'responses': ['Еще увидимся', 'Если что, я всегда тут']
        },
        
        'name': {
             'examples': ['Как тебя зовут?', 'Представься', 'У тебя есть имя?'],
            'responses': ['Меня зовут Федя']
        },
    },
   
    'failure_phrases': [
        'Непонятно. Перефразируйте, пожалуйста',
        'Я еще только учусь. Спросите что-нибудь другое',
        'Слишком сложный вопрос для меня'
    ]
}

In [18]:
#разделение выборки на интенты и классы
X_text = []
y = []
 
for intent, intent_data in BOT_CONFIG['intents'].items():
    for example in intent_data['examples']:
        X_text.append(example)
        y.append(intent)
        
# векторизация
vectorizer = TfidfVectorizer(analyzer = 'char', ngram_range = (3,3))

X = vectorizer.fit_transform(X_text)

# ML
clf = LinearSVC()
clf.fit(X, y)

LinearSVC()

In [19]:
# очистка текста от лишних символов и знаков препинания
def clean(text):
    clean_text = ''
    for ch in text.lower():
        if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя ':
            clean_text += ch
    # с помощью .strip() уберем лишние пробелы в начале и в конце фразы
    return clean_text.strip()

In [20]:
def classify_intent(replica):
    replica = clean(replica)
    intent = clf.predict(vectorizer.transform([replica]))[0]
    
    for example in BOT_CONFIG['intents'][intent]['examples']:
        for example in intent_data['examples']:
            example = clean(example)
            #научимся работать с мелкими ошибками с помощью расстояния Левенштейна
            distance = nltk.edit_distance(replica, example)
            if distance / len(example) <= 0.5:
                return intent
classify_intent('как тебя зовут?')

'name'

In [21]:
def get_answer_by_intent(intent):
      for intent in BOT_CONFIG['intents']:
            responses = BOT_CONFIG['intents'][intent]['responses']
            return random.choice(responses)
        
# в качестве корпуса текстов возьмем корпус из 
# https://github.com/Koziev/NLP_Datasets/blob/master/Conversations/Data/dialogues.zip

with open('/content/dialogues.txt', encoding="utf8") as f:
    content = f.read()
    
content[:1000]

# запишем каждый диалог с новой строки и отделим фразы \n\n
dialogues_str = content.split('\n\n')
# уберем лишнюю \n, оставим в диалоге только 2 фразы и выведем результат
dialogues = [dialogues_str.split('\n')[:2] for dialogues_str in dialogues_str]

dialogues_filtered = []

# убрали дубликаты диалогов
questions = set()

for dialogue in dialogues:
    if len(dialogue) != 2:
        continue
    
    question, answer = dialogue
    #убираем перевые два символа из фразы
    question = clean(question[2:])
    answer = answer[2:]
    
    if question != '' and question not in questions:
        questions.add(question)
        dialogues_filtered.append([question, answer])

# структурируем вопросы по словам для повышения скорости вычислений
dialogues_structured = {}

for question, answer in dialogues_filtered:
    words = question.split(' ')
    
    for word in words:
        if word not in dialogues_structured:
            dialogues_structured[word] = []
        dialogues_structured[word].append([question, answer])

# # у слишком частотных слов оставим только первую 1000 -> на каждую пару теперь приходится не более 1000 элементов
dialogues_structured_cut = {}

for word, pairs in dialogues_structured.items():
    pairs.sort(key=lambda pair: len(pair[0]))
    dialogues_structured_cut[word] = pairs[:1000]
    
# генерируем ответ    
def generate_answer(replica):
    
    replica = clean(replica)
    words = set(replica.split(' '))
    
    mini_dataset = []
    for word in words:
        if word in dialogues_structured:
            mini_dataset += dialogues_structured_cut[word]     
    
    print(len(mini_dataset))
    
    answers = []
    
    for question, answer in mini_dataset:
        # чтобы не считать расстоянение заведомо далеких фраз, отфильтруем их
        abs(len(replica) - len(question)) < 0.2
        #научимся работать с мелкими ошибками с помощью расстояния Левинштейна
        distance = nltk.edit_distance(replica, question)
        distance_weighted = distance / len(question)
        if distance_weighted < 0.2:
            answers.append([distance_weighted, question, answer])
            
        # из всех подходящих вариантов выберем не первый, а наиболее близкий по distance 
    if answers:
        return min(answers, key=lambda three: three[0])[2]
    
#зададим вопрос боту            
generate_answer('мальчик или девочка?')

2411


'Мальчик, Точно.'

In [22]:
def get_failure_phrase():
    failure_phrases = [
      "Слишком ложно и непонятно, напиши чутка иначе",
      "Я понял, что я не понял",
      "что-то непонятно",
      "Краткость - сестра таланта, скажите короче",
      "Давай еще разочек",
      "Пожалуйста, уточните",
      "Не могли бы вы уточнить вопрос?",
    ]
    return random.choice(failure_phrases)

In [23]:
# проверим статистику пути выбора ответа
stats = {'intent': 0, 'generate': 0, 'failure': 0}

In [24]:
def bot(replica):
    # NLU
    intent = classify_intent(replica)
    
    # answer generation
   
    #выбор заготовленной реплики - ML
    if intent:
            answer = get_answer_by_intent(intent)
            if answer:
                stats['intent'] += 1
                return answer
    
    # вызов генеративной модели
    answer = generate_answer(replica)
    if answer:
        stats['generate'] += 1
        return answer
    
    # берем заглушку
    stats['failure'] += 1
    return get_failure_phrase()

In [25]:
bot('первый закон термодинамики')

973


'Пожалуйста, уточните'

In [26]:
stats

{'intent': 0, 'generate': 0, 'failure': 1}

In [27]:
# приступаем к настройке бота из https://github.com/python-telegram-bot/python-telegram-bot
!pip install python-telegram-bot --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import tornado
tornado.version

'6.2'

In [29]:
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext


def start(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def run_bot(update: Update, _: CallbackContext) -> None:
    """Echo the user message."""
    replica = update.message.text
    answer = bot(replica)
    update.message.reply_text(answer)

    print(stats)
    print(replica)
    print(answer)
    print()
    
def main() -> None:
    """Start the bot."""
    updater = Updater("5527195665:AAFnReGBzkHKBqSfbMV2Zl2d14xO-ww-iIk")
    dispatcher = updater.dispatcher
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, run_bot))

    # Start the Bot
    updater.start_polling()

    updater.idle()

In [30]:
main()

1000
{'intent': 0, 'generate': 1, 'failure': 1}
Привет
Привет!

2000
{'intent': 0, 'generate': 2, 'failure': 1}
Как дела
Нормально, А почему не гуляем на свадьбе?

{'intent': 1, 'generate': 2, 'failure': 1}
Как тебя зовут?
Привет, человек



# Справка: Подбор модели, параметров и валидация

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

clf = LogisticRegression()
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

scores = []

#vectorizer = CountVectorizer(analyzer = 'char', ngram_range = (3,3))
vectorizer = TfidfVectorizer(analyzer = 'char', ngram_range = (3,3))

X = vectorizer.fit_transform(X_text)

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    
    clf = LinearSVC()
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    scores.append(score)
    
print(scores)
print(sum(scores) / len(scores))

#проверим, какого максимального качества может добиться логистическая модель
clf = LogisticRegression()
clf.fit(X, y)
clf.score(X, y)

clf = LinearSVC()
clf.fit(X, y)
clf.score(X, y)